In [1]:
#6->2->6

## Imports

In [2]:
import random

import pennylane as qml
from pennylane import numpy as np

from pennylane.optimize import AdamOptimizer

import torch
from torchvision import datasets, transforms

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tomut\anaconda3\envs\qhack2022\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import sys
sys.path.append("..") # Adds higher directory to python modules path

from qencode.initialize import setAB_amplitude, setAux, setEnt
from qencode.encoders import e2_classic
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

from qencode.utils.mnist import get_dataset

## Get Data

In [4]:
input_data = get_dataset(img_width=8, img_height=8, train=True)
print("Original data set size:", len(input_data))

# Select only the pictures with numbers 0 or 1. (jus to compare with literature)
filtered_data = [image for image in input_data if image[1] in [0, 1]]
input_data = filtered_data
print("Final data set szize:", len(input_data))

Original data set size: 60000
Final data set szize: 12665


## Training node

In [5]:
shots = 2500
nr_trash=4
nr_latent=2
nr_ent=0

spec = QubitsArrangement(nr_trash, nr_latent, nr_swap=1, nr_ent=nr_ent)
print("Qubits:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=spec.num_qubits)

Qubits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [6]:
@qml.qnode(dev)
def training_circuit_example(init_params, encoder_params, reinit_state):
    #initilaization
    setAB_amplitude(spec, *init_params)

    setAux(spec, reinit_state)

    setEnt(spec, inputs=[1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)])

    #encoder

    for params in encoder_params:
        e2_classic(params, [*spec.latent_qubits, *spec.trash_qubits])

    #swap test 
    swap_t(spec)

    return [qml.probs(i) for i in spec.swap_qubits]

## Training parameters 

In [7]:
epochs = 2000
learning_rate = 0.0003
batch_size = 5
num_samples = 20

beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [8]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [9]:
def cost(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [10]:
def fidelity(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [11]:
def iterate_batches(X, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [12]:
training_data = [input_data[i] for i in range(num_samples)]
test_data = [input_data[i] for i in range(num_samples,num_samples+num_samples)]

In [13]:
X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in training_data]
X2 = []
for i in range(len(X1)):
    X2.append([X1[1], training_data[i][1]])
X_training = X2

X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in test_data]
X2 = []
for i in range(len(X1)):
    X2.append([X1[1], training_data[i][1]])
X_tes = X2

In [14]:
# initialize random encoder parameters
nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)
nr_par_encoder =  15 * int(nr_encod_qubits*(nr_encod_qubits-1)/2)
encoder_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)


###  training

In [ ]:
loss_hist=[]
fid_hist=[]

loss_hist_test=[]
fid_hist_test=[]

for epoch in range(epochs):
    batches = iterate_batches(X=training_data, batch_size=batch_size)
    for xbatch in batches:
        encoder_params = opt.step(cost, encoder_params, X=xbatch)

        
    if epoch%5 == 0:
        
        loss_training = cost(encoder_params, X_training )
        fidel = fidelity(encoder_params, X_training )
        
        loss_hist.append(loss_training)
        fid_hist.append(fidel)
        print("Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_training, fidel))

        loss_test = cost(encoder_params, X_tes )
        fidel = fidelity(encoder_params, X_tes )
        loss_hist_test.append(loss_test)
        fid_hist_test.append(fidel)
        print("Test-Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_test, fidel))

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\pennylane\math\multi_dispatch.py:63: UserWarning: Contains tensors of types {'torch', 'autograd'}; dispatch will prioritize TensorFlow and PyTorch over autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Epoch:0 | Loss:1.9057777332648205 | Fidelity:0.524720161509539
Test-Epoch:0 | Loss:1.8977828352513693 | Fidelity:0.5269306800677993
Epoch:5 | Loss:1.874614982630942 | Fidelity:0.5334428718779057
Test-Epoch:5 | Loss:1.8690206101675695 | Fidelity:0.5350395787825709
Epoch:10 | Loss:1.8382588809283498 | Fidelity:0.5439930199031514
Test-Epoch:10 | Loss:1.8358555561929493 | Fidelity:0.5447051630106021
Epoch:15 | Loss:1.7961254547164096 | Fidelity:0.5567539825094735
Test-Epoch:15 | Loss:1.7978957820386914 | Fidelity:0.5562057656456971
Epoch:20 | Loss:1.7485024332100572 | Fidelity:0.5719179916519246
Test-Epoch:20 | Loss:1.75459499160763 | Fidelity:0.5699320953171991
Epoch:25 | Loss:1.6974662204727615 | Fidelity:0.5891133431341511
Test-Epoch:25 | Loss:1.7069468150094385 | Fidelity:0.5858413344849709
Epoch:30 | Loss:1.6467702255655794 | Fidelity:0.6072492594748925
Test-Epoch:30 | Loss:1.6581491247478766 | Fidelity:0.6030820660669173
Epoch:35 | Loss:1.600156245191726 | Fidelity:0.624938972681498


Test-Epoch:300 | Loss:1.124194904604071 | Fidelity:0.8895254692087301
Epoch:305 | Loss:1.0996792352060214 | Fidelity:0.9093560812873346
Test-Epoch:305 | Loss:1.1220681254756903 | Fidelity:0.8912114846645866
Epoch:310 | Loss:1.0977929332754794 | Fidelity:0.9109185982973177
Test-Epoch:310 | Loss:1.1200538480342355 | Fidelity:0.8928142175977186
Epoch:315 | Loss:1.096009388678881 | Fidelity:0.9124009432121657
Test-Epoch:315 | Loss:1.1181455111521392 | Fidelity:0.8943379819766017
Epoch:320 | Loss:1.0943221439970778 | Fidelity:0.9138076986612363
Test-Epoch:320 | Loss:1.1163368760981065 | Fidelity:0.8957869451515975
Epoch:325 | Loss:1.0927250935762403 | Fidelity:0.9151432559558298
Test-Epoch:325 | Loss:1.1146220126919744 | Fidelity:0.8971651273823797
Epoch:330 | Loss:1.09121247101956 | Fidelity:0.9164118139757539
Test-Epoch:330 | Loss:1.1129952865247907 | Fidelity:0.898476401568953
Epoch:335 | Loss:1.0897788363693564 | Fidelity:0.9176173794414485
Test-Epoch:335 | Loss:1.1114513475664753 | Fid

Epoch:605 | Loss:1.0575223047129259 | Fidelity:0.945606532877299
Test-Epoch:605 | Loss:1.0774690645999816 | Fidelity:0.9281008920393067
Epoch:610 | Loss:1.0571886254592227 | Fidelity:0.9459049936009475
Test-Epoch:610 | Loss:1.077169858659271 | Fidelity:0.9283586910282441
Epoch:615 | Loss:1.056855745594802 | Fidelity:0.9462029270959744
Test-Epoch:615 | Loss:1.0768735878007238 | Fidelity:0.9286141022757167
Epoch:620 | Loss:1.0565233992513519 | Fidelity:0.9465005703693794
Test-Epoch:620 | Loss:1.0765799392566182 | Fidelity:0.928867391575681
Epoch:625 | Loss:1.056191321366504 | Fidelity:0.9467981603050823
Test-Epoch:625 | Loss:1.0762885988169846 | Fidelity:0.9291188265853249
Epoch:630 | Loss:1.0558592472076602 | Fidelity:0.9470959340883869
Test-Epoch:630 | Loss:1.0759992505555709 | Fidelity:0.9293686770540683
Epoch:635 | Loss:1.055526911933068 | Fidelity:0.9473941295998063
Test-Epoch:635 | Loss:1.0757115766216783 | Fidelity:0.9296172149978583
Epoch:640 | Loss:1.0551940501973365 | Fidelity:

Test-Epoch:905 | Loss:1.0589423703910226 | Fidelity:0.9443384531216203
Epoch:910 | Loss:1.0338901336541322 | Fidelity:0.967220759197738
Test-Epoch:910 | Loss:1.058710853430995 | Fidelity:0.9445449593335805
Epoch:915 | Loss:1.0335642965194345 | Fidelity:0.9675256811477875
Test-Epoch:915 | Loss:1.0584851320980317 | Fidelity:0.9447463829915984
Epoch:920 | Loss:1.0332460797799168 | Fidelity:0.967823657470834
Test-Epoch:920 | Loss:1.0582649076896686 | Fidelity:0.9449429842506365
Epoch:925 | Loss:1.0329352901645354 | Fidelity:0.9681148562953158
Test-Epoch:925 | Loss:1.058049891029461 | Fidelity:0.9451350153507609
Epoch:930 | Loss:1.032631723754616 | Fidelity:0.9683994564529076
Test-Epoch:930 | Loss:1.057839806785765 | Fidelity:0.9453227167150086
Epoch:935 | Loss:1.0323351700474215 | Fidelity:0.9686776436707698
Test-Epoch:935 | Loss:1.0576343964512847 | Fidelity:0.9455063142380131
Epoch:940 | Loss:1.0320454152390455 | Fidelity:0.9689496074825129
Test-Epoch:940 | Loss:1.0574334201072266 | Fide

Epoch:1205 | Loss:1.0217603690235095 | Fidelity:0.9787030602446386
Test-Epoch:1205 | Loss:1.049936847421365 | Fidelity:0.9524382370768208
Epoch:1210 | Loss:1.0215870743426012 | Fidelity:0.9788690803899487
Test-Epoch:1210 | Loss:1.0498153416821459 | Fidelity:0.9525484723796038
Epoch:1215 | Loss:1.02141223141737 | Fidelity:0.9790366408794057
Test-Epoch:1215 | Loss:1.049693990517689 | Fidelity:0.9526585929169883
Epoch:1220 | Loss:1.0212357860363117 | Fidelity:0.9792057952466265
Test-Epoch:1220 | Loss:1.0495728544685883 | Fidelity:0.9527685436437021
Epoch:1225 | Loss:1.0210576991777685 | Fidelity:0.9793765825430574
Test-Epoch:1225 | Loss:1.049452007315566 | Fidelity:0.9528782574421278
Epoch:1230 | Loss:1.0208779485969806 | Fidelity:0.979549025791307
Test-Epoch:1230 | Loss:1.0493315370090885 | Fidelity:0.9529876542644488
Epoch:1235 | Loss:1.0206965303702025 | Fidelity:0.9797231304757188
Test-Epoch:1235 | Loss:1.049211546518086 | Fidelity:0.9530966403473172
Epoch:1240 | Loss:1.02051346033911

Test-Epoch:1500 | Loss:1.0471004685436234 | Fidelity:0.9550181955231732
Epoch:1505 | Loss:1.0115838992931596 | Fidelity:0.988548750824075
Test-Epoch:1505 | Loss:1.04713723357996 | Fidelity:0.9549846647904909
Epoch:1510 | Loss:1.011439307558707 | Fidelity:0.9886900702066665
Test-Epoch:1510 | Loss:1.0471744675450583 | Fidelity:0.9549507087814589
Epoch:1515 | Loss:1.011293888182576 | Fidelity:0.9888322392584887
Test-Epoch:1515 | Loss:1.0472119853331763 | Fidelity:0.9549164963785671
Epoch:1520 | Loss:1.0111475134664685 | Fidelity:0.9889753835933878
Test-Epoch:1520 | Loss:1.047249594865978 | Fidelity:0.9548822027741867
Epoch:1525 | Loss:1.0110000641036172 | Fidelity:0.9891196207654346
Test-Epoch:1525 | Loss:1.0472870970127288 | Fidelity:0.9548480095404497
Epoch:1530 | Loss:1.010851431978146 | Fidelity:0.9892650575199655
Test-Epoch:1530 | Loss:1.0473242857193024 | Fidelity:0.9548141045093781
Epoch:1535 | Loss:1.0107015231339296 | Fidelity:0.9894117868737877
Test-Epoch:1535 | Loss:1.047360948

Epoch:1800 | Loss:1.0038906063792086 | Fidelity:0.9961244717756241
Test-Epoch:1800 | Loss:1.046246740175262 | Fidelity:0.9557974821813883
Epoch:1805 | Loss:1.00383508971947 | Fidelity:0.9961795620030156
Test-Epoch:1805 | Loss:1.0462032291914825 | Fidelity:0.9558372332427336
Epoch:1810 | Loss:1.0037818479611433 | Fidelity:0.9962324005272409
Test-Epoch:1810 | Loss:1.0461608249243048 | Fidelity:0.9558759764038717
Epoch:1815 | Loss:1.0037307584717792 | Fidelity:0.9962831083532204


## Rezults

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist),label="train fid")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist_test),label="test fid")


plt.legend()
plt.title("6-2-6:fidelity e2",)
plt.xlabel("epoch")
plt.ylabel("fid")

print("fidelity:",fid_hist[-1])

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist),label="train loss")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist_test),label="test loss")


plt.legend()
plt.title("6-2-6:loss e2",)
plt.xlabel("epoch")
plt.ylabel("loss")

print("loss:",loss_hist[-1])

In [ ]:
name = "training_e2"

Circuit_prop={   "shots":shots, "nr_trash":nr_trash, "nr_latent":nr_latent "nr_ent":nr_ent  }
Training_param = { "num_samples" = num_samples
                    "batch_size" = batch_size
                    "nr_layers"= nr_layers
                    "epochs" = epochs,
                    "learning_rate" = learning_rate 
                    "beta1" = beta1
                    "beta2 "= beta2
                     "optimizer":"Adam"}

performance={"loss_hist":loss_hist, "fid_hist":fid_hist,
             "loss_hist_test":loss_hist_test, "fid_hist_test":fid_hist_test,
             "encoder_params":encoder_params}

experiment_data={"Circuit_prop":Circuit_prop,
                "Training_param":Training_param,
                "performance:":performance,
                "Name":name}

# open file for writing
f = open(name+".txt","w")
f.write( str(experiment_data) )